# Auto-Evaluation Example

## Installations

In [2]:
!pip install --quiet --force-reinstall prompttools

## Setup imports and API keys

First, we'll need to set our API keys. If we are in DEBUG mode, we don't need to use real OpenAI or Hegel AI API keys, so for now we'll set them to empty strings.

In [3]:
import os
os.environ['DEBUG'] = ""  # Set to "1" if you want to use debug mode.
os.environ['HEGELAI_API_KEY'] = ""  # Optional, it will be needed to use with `HegelScribe` to persist/visualize your experiments
os.environ['OPENAI_API_KEY'] = ""

Then we'll import the relevant `prompttools` modules to set up our experiment.

In [4]:
from typing import Dict
from prompttools.harness import PromptTemplateExperimentationHarness

## Run experiments

Next, we create our test inputs. For this example, we'll use a prompt template, which uses [jinja](https://jinja.palletsprojects.com/en/3.1.x/) for templating.

In [54]:
prompt_templates = ["Answer the following question: {{input}}", "Respond the following query: {{input}}"]
user_inputs = [{"input": "Who was the first president?"}, {"input": "Who was the first president of India?"}]

Now we can define an experimentation harness for our inputs and model. We could also pass model arguments if, for example, we wanted to change the model temperature.

In [55]:
harness = PromptTemplateExperimentationHarness("text-davinci-003", prompt_templates, user_inputs)

We can then run the experiment to get results.

In [57]:
harness.run()
harness.visualize()

You can use the `pivot` keyword argument to view results by the template and inputs that created them.

In [58]:
harness.visualize(pivot=True)

## Auto-Evaluate the model response

To evaluate the model response, we can define an eval method that passes the input and response into another LLM to get feedback.

In [ ]:
import openai
import jinja2

ECHO_EVALUATION_TEMPLATE="""
Determine whether or not the response is following directions.
Your answer should either be "RIGHT" if the response follows directions,
or "WRONG" if the model is not following directions.

Write your answer here:
PROMPT: {{prompt}}
RESPONSE: {{response}}
ANSWER: 
"""

def extract_responses(output) -> str:
    return [choice["text"] for choice in output["choices"]]

def auto_eval(prompt: str, results: Dict, metadata: Dict) -> float:
    environment = jinja2.Environment()
    template = environment.from_string(ECHO_EVALUATION_TEMPLATE)
    responses = extract_responses(results)
    prompts = [template.render({"prompt": prompt, 
                                "response": response}) for response in responses]
    evals = [openai.Completion.create(model="text-davinci-003",
                                      prompt=prompt) for prompt in prompts]
    return float(sum([1 if 'RIGHT' in e['choices'][0]['text'] else 0 for e in evals]))

Finally, we can evaluate and visualize the results.

In [ ]:
harness.evaluate("followed_directions", auto_eval, use_input_pairs=False)
harness.visualize()